# Minting Tokens for Marlowe Contracts

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

This notebook demonstrates how to mint Cardano native tokens using Marlowe's CLI tool and how to use such tokens in a Marlowe contract. (Note that Marlowe contracts themselves will not mint tokens, but they can use tokens minted elsewhere.)

[A video works through this Jupyter notebook.](https://youtu.be/S0MOipqXpmQ)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

## Preliminaries

See [Preliminaries](../../docs/preliminaries.md) for information on setting up one's environment for using this tutorial.

The first step is to check we have all the required tools and environment variables available to the notebook. 

In [ ]:
export SCRIPTS=../../scripts
export KEYS=../../keys
source $SCRIPTS/check-tools-and-env.sh

Make sure you've also [setup and funded](../../setup/01-setup-keys.ipynb) the different parties
- Token distributor
    - `$KEYS/lender.address`: Cardano address for the token distributor
    - `$KEYS/lender.skey`: location of signing key file for the token distributor

### Token distributor address and funds

Check that an address and key has been created for the token distributor. If not, see "Creating Addresses and Signing Keys" in [Setup Keys](../../setup/01-setup-keys.ipynb#Creating-Addresses-and-Signing-Keys).

In [ ]:
TOKEN_DISTRIBUTOR_SKEY=$KEYS/lender.skey
TOKEN_DISTRIBUTOR_ADDR=$(cat $KEYS/lender.address)
echo "TOKEN_DISTRIBUTOR_ADDR = $TOKEN_DISTRIBUTOR_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for a transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-address.sh $TOKEN_DISTRIBUTOR_ADDR

## Design the tokens

Now we create [CIP-25](https://cips.cardano.org/cips/cip25/) metadata for the tokens that we will mint. CIP23 metadata has key `721` and consists of the following:

- JSON describing the token.
- Optionally, links to external resources such as an image for the token.

The metadata is stored on the blockchain in the transaction that mints the token, but the images are stored off of the blockchain and just referenced by the on-chain metadata.

We will create a token with asset name `M4B` and associate with it metadata with a description and image.

In [ ]:
TOKEN_NAME=M4B
METADATA_FILE=marlowe-babbage.json
IMAGE_IPFS=QmZqCCHLqQcHXftNarCwKpRHbzF4mvNeQRVpzk2bdue5bw

We're using IPFS to store the image, but the image can be hosted at any URI. Many [IPFS pinning services](https://sourceforge.net/software/ipfs-pinning/) are available. Let's fetch the image, even though the image file is note required for the minting process.

In [ ]:
IMAGE_URL=https://ipfs.io/ipfs/"$IMAGE_IPFS"
IMAGE_FILE=marlowe-babbage.png
curl -sS "$IMAGE_URL" -o "$IMAGE_FILE"

Here is the image itself:

![Marlowe in the Babbage Era](marlowe-babbage.png)

Here is the metadata for the token that we will create.

In [ ]:
yaml2json << EOI > "$METADATA_FILE"
$TOKEN_NAME:
  description: Marlowe smart contracts in the Babbage Era
  image: ipfs://$IMAGE_IPFS
  mediaType: image/gif
  name: Marlowe in the Babbage Era
  url: https://marlowe.iohk.io/
EOI
jq . marlowe-babbage.json

The above only contains metadata for a single token, but the JSON can have entries for many tokens.

The `M4B` key is the asset name for the token on the blockchain. The nested fields describe that token.

- `name` is the human-friendly name for the token.
- `image` is a URI referencing the location of the image for the token.
- `mediaType` is the MIME type for the image.
- `description` is text describing the token.
- `url` is a URL to find more information related to the token.

The [CIP25 standard](https://cips.cardano.org/cips/cip25/#generalstructure) provides information on general structure of the metadata.

## Mint the tokens

The `marlowe-cli` tool supports minting tokens according to the CIP25 standard.

In [ ]:
marlowe-cli util mint --help

The `marlowe-cli util mint` command uses a Cardano Simple Script V2 minting policy. If the `--expires` option is specified, then the minting policy is "locked" after a specified slot, so that no more tokens with that policy can ever be minted at or after that slot number; if not, then there is no time limit on minting new tokens or burning old ones.

First find the slot number for the tip of the blockchain.

In [ ]:
TIP=$(cardano-cli query tip --testnet-magic "$CARDANO_TESTNET_MAGIC" | jq -r .slot)
echo "The tip is at slot number $TIP."

Let's set the expiration slot to five hours into the future. That gives us some time to burn and recreate the tokens if we've made typographical or other errors in the metadata.

In [ ]:
SECONDS=1
MINUTES=$((60 * SECONDS))
HOURS=$((60 * MINUTES))
EXPIRES=$((TIP + 5 * HOURS))
echo "EXPIRES = $EXPIRES"

We'll mint six tokens.

In [ ]:
TOKEN_COUNT=6

We provide the token distributor's key information, the metadata file, the count of tokens, the expiration slot, and the destinations. In return, the tool prints the policy ID for the minting script.

In [ ]:
TOKEN_POLICY=$(
marlowe-cli util mint \
  --issuer "$TOKEN_DISTRIBUTOR_ADDR:$TOKEN_DISTRIBUTOR_SKEY" \
  --metadata-file "$METADATA_FILE" \
  --count "$TOKEN_COUNT" \
  --expires "$EXPIRES" \
  --out-file /dev/null \
  --submit 600s \
  "$TOKEN_NAME:$TOKEN_DISTRIBUTOR_ADDR" \
  2> /dev/null \
)
echo "TOKEN_POLICY = $TOKEN_POLICY"

We can view the minted tokens on an explorer:

In [ ]:
echo "$CARDANO_SCAN_URL/tokenPolicy/$TOKEN_POLICY"

## *Optional:* Burning tokens

Marlowe CLI also provides a command for un-minting (i.e., "burning") previously minted tokens.

In [ ]:
marlowe-cli util burn --help

## *Optional:* Details of the minting script

The Marlowe CLI uses a Simple Script V2 as its minting policy. Let's manually derive that policy and see that it has the same policy ID as the tool reported.

The JSON for the policy script can be used with `cardano-cli`, uploaded to explorers like CardanoScan, etc.

First, we need the public-key hash (PKH) of the signing key used to mint the tokens. In our case this is the token

In [ ]:
TOKEN_DISTRIBUTOR_PKH=$(
cardano-cli key verification-key --signing-key-file "$TOKEN_DISTRIBUTOR_SKEY" --verification-key-file /dev/stdout \
| cardano-cli address key-hash --verification-key-file /dev/stdin \
)
echo "TOKEN_DISTRIBUTOR_PKH = $TOKEN_DISTRIBUTOR_PKH"

### Minting policy with an expiration slot

For a minting policy with an expiration slot, Marlowe CLI generates a monetary policy of the following form:

In [ ]:
yaml2json << EOI > policy-$EXPIRES.json
type: all
scripts:
- type: sig
  keyHash: $TOKEN_DISTRIBUTOR_PKH
- type: before
  slot: $EXPIRES
EOI
jq . policy-$EXPIRES.json

We can verify that this policy corresponds to the policy ID that we used to mint the tokens.

In [ ]:
cardano-cli transaction policyid --script-file "policy-$EXPIRES.json"

### Minting policy without an expiration slot

The minting policy generated by Marlowe CLI is simplier if there is not expiration slot.

In [ ]:
yaml2json << EOI > policy-noexpires.json
type: sig
keyHash: $TOKEN_DISTRIBUTOR_PKH
EOI
jq . policy-noexpires.json

Once again, we can verify the policy ID of such.

In [ ]:
cardano-cli transaction policyid --script-file policy-noexpires.json

## *Optional:* Example Marlowe contract: a small airdrop to Ada Handle holders

The tokens that we've minted can be used in ordinary transactions or in Marlowe contracts. Here we'll use them in a small airdrop to six holders of [Ada Handle](https://mint.handle.me/) tokens.

This example illustrates how a well-known monetary policy like that for [Ada Handles](https://mint.handle.me/) or [Ada Domains](https://www.adadomains.io/) can be used as the roles currency symbol for a Marlowe contract. This can be convenient because Marlowe's payout validator provides *an on-chain guarantee* that funds will be delivered to the holder of the role token: that is, no off-chain services are needed to locate the address of the handle's or domain's holder.

The below contract accepts a deposit of the six tokens we just created and then distribute one each to the holders of six Ada Handles:
- `$e.cary`
- `$f.beaumont`
- `$j.lumley`
- `$j.webster`
- `$m.herbert`
- `$w.shakespeare`

![Small airdrop contract in Marlowe](contract.png)

Note the presence of the two `Notify` cases in the contract. These are necessary to break up the payments into three transactions. Attempting to perform them all in the same transaction as the deposit would result in Plutus execution costs being exceeded.

### Design the contract

First, set the deadline to deposit the tokens twenty minutes into the future, expressed in the POSIX milliseconds.

In [ ]:
DEPOSIT_DEADLINE=$((1000 * $(date -u -d "$(date) + 20 minutes" +%s)))
echo "DEPOSIT_DEADLINE = $DEPOSIT_DEADLINE POSIX milliseconds"

Write the contract to a file. (We could have just created the contract in [Marlowe Playground](https://play.marlowe.iohk.io/) and then downloaded the JSON.) *Note that the role names do not contain the `$` that prefixes an Ada Handle; similarly, they would not include the `.ada` suffix of an Ada Domain.*

In [ ]:
yaml2json << EOI > contract.json
when:
- case:
    deposits: 6
    into_account:
      address: $TOKEN_DISTRIBUTOR_ADDR
    of_token:
      currency_symbol: $TOKEN_POLICY
      token_name: $TOKEN_NAME
    party:
      address: $TOKEN_DISTRIBUTOR_ADDR
  then:
    from_account:
      address: $TOKEN_DISTRIBUTOR_ADDR
    pay: 1
    token:
      currency_symbol: $TOKEN_POLICY
      token_name: $TOKEN_NAME
    to:
      party:
        role_token: e.cary
    then:
      from_account:
        address: $TOKEN_DISTRIBUTOR_ADDR
      pay: 1
      token:
        currency_symbol: $TOKEN_POLICY
        token_name: $TOKEN_NAME
      to:
        party:
          role_token: f.beaumont
      then:
        when:
        - case:
            notify_if: true
          then:
            from_account:
              address: $TOKEN_DISTRIBUTOR_ADDR
            pay: 1
            token:
              currency_symbol: $TOKEN_POLICY
              token_name: $TOKEN_NAME
            to:
              party:
                role_token: j.lumley
            then:
              from_account:
                address: $TOKEN_DISTRIBUTOR_ADDR
              pay: 1
              token:
                currency_symbol: $TOKEN_POLICY
                token_name: $TOKEN_NAME
              to:
                party:
                  role_token: j.webster
              then:
                when:
                - case:
                    notify_if: true
                  then:
                    from_account:
                      address: $TOKEN_DISTRIBUTOR_ADDR
                    token:
                      currency_symbol: $TOKEN_POLICY
                      token_name: $TOKEN_NAME
                    to:
                      party:
                        role_token: m.herbert
                    pay: 1
                    then:
                      from_account:
                        address: $TOKEN_DISTRIBUTOR_ADDR
                      token:
                        currency_symbol: $TOKEN_POLICY
                        token_name: $TOKEN_NAME
                      pay: 1
                      to:
                        party:
                          role_token: w.shakespeare
                      then: close
                timeout: $DEPOSIT_DEADLINE
                timeout_continuation: close
        timeout: $DEPOSIT_DEADLINE
        timeout_continuation: close
timeout: $DEPOSIT_DEADLINE
timeout_continuation: close
EOI
cat contract.json

### Transaction 1. Create the contract on the blockchain

We'll be sending the tokens to holders of [ADA Handles](https://mint.handle.me/), so we need to use the Ada Handles policy ID as the Marlowe roles currency symbol.

⚠️ *If you are not running on `mainnet`, then you'll need to use a different policy ID for the role tokens and you must mint those yourself (or use pre-existing role tokens).*

In [ ]:
ADA_HANDLES_POLICY=f0ff48bbb7bbe9d59a40f1ce90e9e9d0ff5002ec48f232b49ca0fb9a

Now have Marlowe Runtime build the transaction to create the Marlowe contract. See [Lesson 1](../01-runtime-cli) for a more detailed exposition.

In [ ]:
CONTRACT_ID=$(
marlowe-runtime-cli create \
  --core-file contract.json \
  --role-token-policy-id "$ADA_HANDLES_POLICY" \
  --min-utxo "$((2 * 1000000))" \
  --change-address "$TOKEN_DISTRIBUTOR_ADDR" \
  --manual-sign tx-1.unsigned \
| jq -r 'fromjson | .contractId' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

*Always check that the contract has no safety errors before submitting the transaction that creates it.* See [Lesson 7](../07-safety) for a detailed discussion of the safety checks that Marlowe Runtime does and why they are important.

Since the contract is safe, we sign and submit the transaction.

In [ ]:
marlowe-cli transaction submit \
  --required-signer "$TOKEN_DISTRIBUTOR_SKEY" \
  --tx-body-file tx-1.unsigned \
  --timeout 600s

After the transaction is confirmed, we can view the contract on MarloweScan:

In [ ]:
echo "$MARLOWE_SCAN_URL/contractView?tab=info&contractId=${CONTRACT_ID/\#/%23}"

### Transaction 2. Deposit the tokens and make the first two airdrops

First build the transaction that will deposit the tokens in the contract and make the payments to `$e.cary` and `$f.beaumont`.

In [ ]:
TX_2=$(
marlowe-runtime-cli deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$TOKEN_DISTRIBUTOR_ADDR" \
  --to-party "$TOKEN_DISTRIBUTOR_ADDR" \
  --currency "$TOKEN_POLICY" \
  --token-name "$TOKEN_NAME" \
  --quantity "$TOKEN_COUNT" \
  --change-address "$TOKEN_DISTRIBUTOR_ADDR" \
  --manual-sign tx-2.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_2 = $TX_2"

Now sign and submit it.

In [ ]:
marlowe-cli transaction submit \
  --required-signer "$TOKEN_DISTRIBUTOR_SKEY" \
  --tx-body-file tx-2.unsigned \
  --timeout 600s

After the transaction is confirmed, we can view it on MarloweScan.

In [ ]:
echo "$MARLOWE_SCAN_URL/contractView?tab=tx&contractId=${CONTRACT_ID/\#/%23}&transactionId=$TX_2"

### Transaction 3. Notify the contract to make the next two airdrops

Now build the transaction that will make the payments to `$j.lumley` and `$j.webster`.

In [ ]:
TX_3=$(
marlowe-runtime-cli notify \
  --contract "$CONTRACT_ID" \
  --change-address "$TOKEN_DISTRIBUTOR_ADDR" \
  --manual-sign tx-3.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_3 = $TX_3"

Now sign and submit it.

In [ ]:
marlowe-cli transaction submit \
  --required-signer "$TOKEN_DISTRIBUTOR_SKEY" \
  --tx-body-file tx-3.unsigned \
  --timeout 600s

After the transaction is confirmed, we can view it on MarloweScan.

In [ ]:
echo "$MARLOWE_SCAN_URL/contractView?tab=tx&contractId=${CONTRACT_ID/\#/%23}&transactionId=$TX_3"

### Transaction 4. Notify the contract to make the last two airdrops

Now build the transaction that will make the payments to `$m.herbert` and `$w.shakespeare`.

In [ ]:
TX_4=$(
marlowe-runtime-cli notify \
  --contract "$CONTRACT_ID" \
  --change-address "$TOKEN_DISTRIBUTOR_ADDR" \
  --manual-sign tx-4.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_4 = $TX_4"

Now sign and submit it.

In [ ]:
marlowe-cli transaction submit \
  --required-signer "$TOKEN_DISTRIBUTOR_SKEY" \
  --tx-body-file tx-4.unsigned \
  --timeout 600s

After the transaction is confirmed, we can view it on MarloweScan.

In [ ]:
echo "$MARLOWE_SCAN_URL/contractView?tab=tx&contractId=${CONTRACT_ID/\#/%23}&transactionId=$TX_4"

### Recipients withdraw the tokens from the Marlowe role-payout address

The six payments are waiting at Marlowe's role-payout address for the holders of each Ada Handle to withdraw them. Currently there are several methods to withdraw such payments:

1. Connect one's CIP30 wallet to the Marlowe Payouts dapp, which will list all of the payouts held for the benefit the the wallet's owner.
2. Use the withdraw endpoint of the Marlowe Runtime REST API.
3. Use the withdraw command of the Marlowe Runtime CLI.
4. Use the withdraw command of the Marlowe CLI.
5. Craft a withdrawal transaction using Cardano CLI.

## Optional: Use cases involving native tokens and Marlowe contracts

Although Marlowe Runtime does not yet provide support for minting native tokens in conjunction with a Marlowe contract, this can be accomplished using `cardano-cli transaction build`.

1. Tokens can be minted in the same transaction that creates the Marlowe contract by running a Simple or Plutus minting script in that transaction.
2. Tokens can be minted in the same transaction as a Marlowe `Deposit`by running a Simple or Plutus minting script in that transaction.

Note that running a Simple script in the same transaction as Marlowe is inexpensive, but less powerful than running a Plutus script. The Plutus script can enforce stronger assurances about the transaction, such as ensuring that the minting *only* can take place at a particular stage of the Marlowe contract.